# Ejercicio 3: Visualización de Atención con BertViz

## Parte A: Instalación y Configuración

In [1]:
# Instalación de paquetes necesarios
%pip install bertviz transformers torch --quiet

Note: you may need to restart the kernel to use updated packages.


In [2]:
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np

# Cargar modelo y tokenizer
model_name = "bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name, output_attentions=True)

print("Modelo cargado correctamente")
print(f"Capas: {model.config.num_hidden_layers}")
print(f"Cabezas de atención por capa: {model.config.num_attention_heads}")
print(f"Dimensión del modelo: {model.config.hidden_size}")

c:\Users\alvar\OneDrive\Universidad\Data\2º Cuatri\AA 2\ejercicios\tema5\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\alvar\OneDrive\Universidad\Data\2º Cuatri\AA 2\ejercicios\tema5\.venv\Lib\site-packages\huggingface_hub\file_download.py:130: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\alvar\.cache\huggingface\hub\models--bert-base-multilingual-cased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either nee

Modelo cargado correctamente
Capas: 12
Cabezas de atención por capa: 12
Dimensión del modelo: 768


## Parte B: Función de análisis de atención

In [5]:
def analyze_attention(sentence, model, tokenizer):
    """Analiza los patrones de atención para una oración."""
    # Tokenizar
    inputs = tokenizer(sentence, return_tensors="pt")
    tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
    
    # Forward pass
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Extraer atención
    attentions = outputs.attentions
    
    print(f"Oración: {sentence}")
    print(f"Tokens: {tokens}")
    print(f"Número de capas: {len(attentions)}")
    print(f"Forma de atención por capa: {attentions[0].shape}")
    
    return attentions, tokens

def print_attention_weights(attentions, tokens, layer, head, threshold=0.15):
    """Imprime los pesos de atención de una cabeza específica."""
    att = attentions[layer][0, head].numpy()
    print(f"\n=== Capa {layer}, Cabeza {head} ===")
    print(f"{'':>15}", end="")
    for t in tokens:
        print(f"{t:>12}", end="")
    print()
    for i, token in enumerate(tokens):
        print(f"{token:>15}", end="")
        for j in range(len(tokens)):
            val = att[i][j]
            # Resaltar valores altos con *
            marker = "*" if val > threshold else " "
            print(f"{val:>11.4f}{marker}", end="")
        print()

def find_strong_connections(attentions, tokens, source_word, target_word):
    """Encuentra capas y cabezas donde source_word atiende fuertemente a target_word."""
    connections = []
    
    # Buscar índices de los tokens (búsqueda flexible con subpalabras)
    source_indices = []
    target_indices = []
    
    for i, t in enumerate(tokens):
        # Eliminar ## y convertir a minúsculas para búsqueda
        clean_token = t.replace("##", "").lower()
        if source_word.lower() in clean_token or clean_token in source_word.lower():
            source_indices.append(i)
        if target_word.lower() in clean_token or clean_token in target_word.lower():
            target_indices.append(i)
    
    if not source_indices or not target_indices:
        print(f"Source '{source_word}': {source_indices}, Target '{target_word}': {target_indices}")
        return []
    
    print(f"Tokens de '{source_word}': {[tokens[i] for i in source_indices]}")
    print(f"Tokens de '{target_word}': {[tokens[i] for i in target_indices]}")
    
    # Buscar en todas las capas y cabezas
    for layer_idx, layer_att in enumerate(attentions):
        for head_idx in range(layer_att.shape[1]):
            att = layer_att[0, head_idx].numpy()
            for src_idx in source_indices:
                for tgt_idx in target_indices:
                    weight = att[src_idx, tgt_idx]
                    if weight > 0.15:  # Umbral de atención fuerte
                        connections.append({
                            'layer': layer_idx,
                            'head': head_idx,
                            'weight': weight,
                            'source_idx': src_idx,
                            'target_idx': tgt_idx,
                            'source_token': tokens[src_idx],
                            'target_token': tokens[tgt_idx]
                        })
    
    # Ordenar por peso descendente
    connections.sort(key=lambda x: x['weight'], reverse=True)
    return connections

## Parte C: Análisis de Patrones

### Oración 1 - Correferencia

In [6]:
# Oración 1: Correferencia
sentence_1 = "El gato se sentó en la alfombra porque estaba cansado"
attentions_1, tokens_1 = analyze_attention(sentence_1, model, tokenizer)

# Buscar conexiones entre "estaba"/"cansado" y "gato"
print("\n--- Buscando correferencia: estaba/cansado -> gato ---")
connections_estaba = find_strong_connections(attentions_1, tokens_1, "estaba", "gato")
connections_cansado = find_strong_connections(attentions_1, tokens_1, "cansado", "gato")

print("\nTop 5 conexiones 'estaba' -> 'gato':")
for conn in connections_estaba[:5]:
    print(f"Capa {conn['layer']:2d}, Cabeza {conn['head']:2d}: {conn['weight']:.4f}")

print("\nTop 5 conexiones 'cansado' -> 'gato':")
for conn in connections_cansado[:5]:
    print(f"Capa {conn['layer']:2d}, Cabeza {conn['head']:2d}: {conn['weight']:.4f}")

Oración: El gato se sentó en la alfombra porque estaba cansado
Tokens: ['[CLS]', 'El', 'ga', '##to', 'se', 'sent', '##ó', 'en', 'la', 'al', '##fo', '##mbra', 'porque', 'estaba', 'can', '##sado', '[SEP]']
Número de capas: 12
Forma de atención por capa: torch.Size([1, 12, 17, 17])

--- Buscando correferencia: estaba/cansado -> gato ---
Tokens de 'estaba': ['estaba']
Tokens de 'gato': ['ga', '##to']
Tokens de 'cansado': ['can', '##sado']
Tokens de 'gato': ['ga', '##to']

Top 5 conexiones 'estaba' -> 'gato':
Capa  8, Cabeza  2: 0.3982

Top 5 conexiones 'cansado' -> 'gato':
Capa  8, Cabeza  2: 0.3475
Capa  8, Cabeza  2: 0.3121
Capa 10, Cabeza  1: 0.2259
Capa  8, Cabeza  2: 0.1919
Capa  8, Cabeza  7: 0.1786


### Oración 2 - Estructura sintáctica

In [7]:
# Oración 2: Estructura sintáctica
sentence_2 = "Los estudiantes que aprobaron el examen celebraron con sus amigos"
attentions_2, tokens_2 = analyze_attention(sentence_2, model, tokenizer)

# Buscar conexiones entre "celebraron" y "estudiantes"
print("\n--- Buscando relación sintáctica: celebraron -> estudiantes ---")
connections_2 = find_strong_connections(attentions_2, tokens_2, "celebraron", "estudiantes")

print("\nTop 5 conexiones 'celebraron' -> 'estudiantes':")
for conn in connections_2[:5]:
    print(f"Capa {conn['layer']:2d}, Cabeza {conn['head']:2d}: {conn['weight']:.4f}")

Oración: Los estudiantes que aprobaron el examen celebraron con sus amigos
Tokens: ['[CLS]', 'Los', 'estudiantes', 'que', 'ap', '##ro', '##bar', '##on', 'el', 'examen', 'celebrar', '##on', 'con', 'sus', 'amigos', '[SEP]']
Número de capas: 12
Forma de atención por capa: torch.Size([1, 12, 16, 16])

--- Buscando relación sintáctica: celebraron -> estudiantes ---
Tokens de 'celebraron': ['##ro', '##on', 'el', 'celebrar', '##on']
Tokens de 'estudiantes': ['estudiantes']

Top 5 conexiones 'celebraron' -> 'estudiantes':
Capa  0, Cabeza  1: 0.6613
Capa  8, Cabeza  2: 0.5466
Capa  9, Cabeza  4: 0.4172
Capa 10, Cabeza  1: 0.4094
Capa  0, Cabeza  2: 0.3427


### Oración 3 - Relaciones a larga distancia

In [8]:
# Oración 3: Relaciones a larga distancia
sentence_3 = "La empresa que fundaron en Madrid hace diez años finalmente cerró"
attentions_3, tokens_3 = analyze_attention(sentence_3, model, tokenizer)

# Buscar conexiones entre "cerró" y "empresa"
print("\n--- Buscando relación a larga distancia: cerró -> empresa ---")
connections_3 = find_strong_connections(attentions_3, tokens_3, "cerró", "empresa")

print("\nTop 5 conexiones 'cerró' -> 'empresa':")
for conn in connections_3[:5]:
    print(f"Capa {conn['layer']:2d}, Cabeza {conn['head']:2d}: {conn['weight']:.4f}")

if len(connections_3) < 5:
    print(f"\nSolo se encontraron {len(connections_3)} conexiones fuertes (>0.2)")
    print("Las relaciones a larga distancia son más difíciles de capturar cuando hay cláusulas relativas intermedias.")

Oración: La empresa que fundaron en Madrid hace diez años finalmente cerró
Tokens: ['[CLS]', 'La', 'empresa', 'que', 'fundar', '##on', 'en', 'Madrid', 'hace', 'diez', 'años', 'finalmente', 'ce', '##rr', '##ó', '[SEP]']
Número de capas: 12
Forma de atención por capa: torch.Size([1, 12, 16, 16])

--- Buscando relación a larga distancia: cerró -> empresa ---
Tokens de 'cerró': ['ce', '##rr', '##ó']
Tokens de 'empresa': ['empresa']

Top 5 conexiones 'cerró' -> 'empresa':
Capa  7, Cabeza  5: 0.6073
Capa  7, Cabeza  5: 0.5093
Capa  9, Cabeza  4: 0.4473
Capa  8, Cabeza  2: 0.4168
Capa  8, Cabeza  2: 0.4126


### Oración 4 - Comparación de idiomas

In [9]:
# Oración 4: Comparación de idiomas
sentence_es = "El banco está cerca del río"
sentence_en = "The bank is near the river"

attentions_es, tokens_es = analyze_attention(sentence_es, model, tokenizer)
print("\n" + "="*60)
attentions_en, tokens_en = analyze_attention(sentence_en, model, tokenizer)

# Analizar "banco"/"bank"
print("\n--- Analizando 'banco' en español ---")
banco_idx = [i for i, t in enumerate(tokens_es) if "banco" in t.lower()]
if banco_idx:
    print(f"Índice de 'banco': {banco_idx[0]}")
    print("\nPatrones de atención del token 'banco' en capa 6, cabeza 4:")
    print_attention_weights(attentions_es, tokens_es, 6, 4)

print("\n--- Analizando 'bank' en inglés ---")
bank_idx = [i for i, t in enumerate(tokens_en) if "bank" in t.lower()]
if bank_idx:
    print(f"Índice de 'bank': {bank_idx[0]}")
    print("\nPatrones de atención del token 'bank' en capa 6, cabeza 4:")
    print_attention_weights(attentions_en, tokens_en, 6, 4)

Oración: El banco está cerca del río
Tokens: ['[CLS]', 'El', 'banco', 'está', 'cerca', 'del', 'río', '[SEP]']
Número de capas: 12
Forma de atención por capa: torch.Size([1, 12, 8, 8])

Oración: The bank is near the river
Tokens: ['[CLS]', 'The', 'bank', 'is', 'near', 'the', 'river', '[SEP]']
Número de capas: 12
Forma de atención por capa: torch.Size([1, 12, 8, 8])

--- Analizando 'banco' en español ---
Índice de 'banco': 2

Patrones de atención del token 'banco' en capa 6, cabeza 4:

=== Capa 6, Cabeza 4 ===
                      [CLS]          El       banco        está       cerca         del         río       [SEP]
          [CLS]     0.0433      0.1753*     0.1935*     0.0695      0.0701      0.0918      0.2114*     0.1451 
             El     0.0603      0.2836*     0.3254*     0.0743      0.0313      0.0158      0.0880      0.1214 
          banco     0.0760      0.3610*     0.1910*     0.1301      0.0518      0.0191      0.0812      0.0900 
           está     0.0084      0.4312

## Análisis general de patrones por capa

In [10]:
# Comparar patrones en capas tempranas vs profundas
print("=== CAPAS TEMPRANAS (0-3) ===")
for layer in [0, 2]:
    print(f"\nCapa {layer}, Cabeza 0:")
    print_attention_weights(attentions_1, tokens_1, layer, 0, threshold=0.2)

print("\n\n=== CAPAS PROFUNDAS (9-11) ===")
for layer in [9, 11]:
    print(f"\nCapa {layer}, Cabeza 0:")
    print_attention_weights(attentions_1, tokens_1, layer, 0, threshold=0.2)

=== CAPAS TEMPRANAS (0-3) ===

Capa 0, Cabeza 0:

=== Capa 0, Cabeza 0 ===
                      [CLS]          El          ga        ##to          se        sent         ##ó          en          la          al        ##fo      ##mbra      porque      estaba         can      ##sado       [SEP]
          [CLS]     0.1135      0.0070      0.0012      0.0008      0.0027      0.0015      0.0105      0.0067      0.0018      0.0020      0.0011      0.0021      0.0080      0.0507      0.0054      0.0050      0.7799*
             El     0.0222      0.3850*     0.0084      0.0417      0.0679      0.0221      0.0282      0.0472      0.1077      0.0539      0.0124      0.0248      0.0517      0.0316      0.0254      0.0168      0.0530 
             ga     0.0310      0.0582      0.1371      0.0213      0.0467      0.0217      0.0232      0.0799      0.0570      0.0263      0.0284      0.0543      0.0746      0.1509      0.0484      0.0341      0.1069 
           ##to     0.1205      0.0273      0